In [13]:
from scprint import scPrint
from scprint.tasks import GNInfer

from bengrn import BenGRN
import scanpy as sc

from bengrn.base import train_classifier

from anndata.utils import make_index_unique
from bengrn import compute_genie3
from grnndata import utils as grnutils
import numpy as np
import joblib
from huggingface_hub import hf_hub_download

from scdataloader import Preprocessor
%load_ext autoreload
%autoreload 2 

import torch
torch.set_float32_matmul_precision('medium') 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
model_checkpoint_file = hf_hub_download(
    repo_id="jkobject/scPRINT", filename="v2-medium.ckpt"
)

if torch.cuda.is_available():
    print("CUDA is available, using GPU", flush=True)
    precision = "16"
    transformer = "flash"
else:
    print("CUDA is not available, using CPU", flush=True)
    precision = "32"
    transformer = "normal"

m = torch.load(model_checkpoint_file, map_location=torch.device("cpu"))
if "label_counts" in m["hyper_parameters"]:
    model = scPrint.load_from_checkpoint(
        model_checkpoint_file,
        transformer=transformer,  # Don't use this for GPUs with flashattention
        precpt_gene_emb=None,
        classes=m["hyper_parameters"]["label_counts"],
    )
else:
    model = scPrint.load_from_checkpoint(
        model_checkpoint_file,
        transformer=transformer,  # Don't use this for GPUs with flashattention
        precpt_gene_emb=None,
    )
del m

CUDA is available, using GPU
FYI: scPrint is not attached to a `Trainer`.


raw counts - pas normalisé, ni log transformed

15 cell types sous "celltype",  avec 100 cells ou moins par cell type

14  batches sous "rep". Je me disais  pas que ca valait pas le coup de traiter chaque batch séparément car ça en fait beaucoup pour la taille du dataset, mais point très ouvert si tu pense que ça vaut le coup 
 
network en sortie:
Si possible au moins 15k genes conservés, je suis plus intéressé par la qualité et la densité que la différence entre les cell types.

In [11]:
adata.obs['organism_ontology_term_id'] = 'NCBITaxon:9606'

In [14]:
preprocessor = Preprocessor(
    min_valid_genes_id=min(0.9 * adata.n_vars, 10000),  # 90% of features up to 10,000
    # Turn off cell filtering to return results for all cells
    filter_cell_by_counts=300,
    min_nnz_genes=300,
    do_postp=False,
    # Skip ontology checks
    skip_validate=True,
    is_symbol=True,


)
adata = preprocessor(adata)

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 1296 renamining
filtered out 0 cells, 1296 renamining
No ENS genes found, assuming gene symbols...
> /home/ml4ig1/Documents code/scDataLoader/scdataloader/preprocess.py(276)__call__()
    274         pdb.set_trace()
    275 
--> 276         adata = adata[:, var.index]
    277         #        var = var.sort_values(by="ensembl_gene_id").set_index("ensembl_gene_id")
    278         # Update adata with combined genes

Removed 0 genes.
startin QC
Seeing 235 outliers (18.13% of total dataset):
done
AnnData object with n_obs × n_vars = 1296 × 70704
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'channel', 'nCount_HTO', 'nFeature_HTO', 'MULTI_ID', 'MULTI_classification', 'percent_mito', 'RNA_snn_res.1.5', 'seurat_clusters', 'cluster_manual', 'celltype', 'channel_hashtag', 'sample', 'rep', 'organism_ontology_term_id', 'nnz', 'n_counts', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts'

In [7]:
adata = sc.read_h5ad('../data/temp/rna_pbs_immune_dictionary.h5ad')
adata.var["isTF"] = False
adata.var.loc[adata.var.symbol.isin(grnutils.TF), "isTF"] = True
adata

In [ ]:
metrics = {}
clf_omni = None
shapes = []
for celltype in adata.obs['celltype'].unique():
    grn_inferer = GNInfer(model, adata[adata.X.sum(1) > 500],
                        how="random expr",
                        preprocess="softmax",
                        head_agg='mean',
                        filtration="none",
                        forward_mode="none",
                        num_genes=3000,
                        max_cells=5000,
                        doplot=False,
                        batch_size=32,
                        cell_type_col="celltype",
                        )
    grn = grn_inferer(layer=list(range(model.nlayers))[:], cell_type=celltype)
    grn.var.index = make_index_unique(grn.var['symbol'].astype(str))
    metrics[celltype+'_scprint_full'] = BenGRN(grn).scprint_benchmark()
    del grn